In [16]:
import re
# nltk?
from english_words import get_english_words_set

wl = 5  # word length
# getting all wl-len words for wordle
words = [word for word in get_english_words_set(['web2'], lower=True) if len(word) == wl]
print("found", len(words), "words")
words = ' '.join(words)

# TODO: add word frequency --> https://www.kaggle.com/datasets/rtatman/english-word-frequency

found 9979 words


In [3]:
def match_words(to_find=".....", bad_c="", good_c="", guesses=[], verbose=False):
    """ gets a list of all wl-letter words given criteria 
        to_find => the word matches these letters in this order with '.' for unknown
        bad_c => the word doesnt contain these characters
        good_c => the word does contain these characters"""
    # get all words that match this sequence
    found_words = re.findall(to_find, words)
#     if verbose: 
#         print("Found words:", found_words)

#     print("hound" in found_words)  # TODO figure why this isnt working

    # character subset
    bad_c += ' '  # addition for space delim in regex
    res = []
    for word in found_words:
        can_add = True
        for c in bad_c:  # subtract words with bad characters
            if c in word:
                can_add = False
        if can_add:
            for c in good_c:  # add words with good characters
                if c not in word:
                    can_add = False
        if can_add and word in guesses:
            can_add = False  # TODO: elim words with - feedback matching letters
        if can_add:
            res.append(word)
    if verbose:
        print("\nResulting words:")
        print(res)
    return res

potential_words = match_words(".....", "", "", [], True)  # easiest


Resulting words:
['pupal', 'hider', 'pried', 'niall', 'brush', 'igdyr', 'kieye', 'sorra', 'sided', 'slank', 'skirp', 'dolor', 'akali', 'jenna', 'super', 'pudgy', 'skulk', 'banig', 'doter', 'abram', 'daler', 'linje', 'abies', 'spoke', 'slath', 'carlo', 'lethe', 'ticca', 'regal', 'inurn', 'hache', 'venus', 'slive', 'teras', 'gully', 'udder', 'amice', 'idler', 'upbid', 'peery', 'birch', 'tepee', 'bandi', 'cagit', 'casel', 'tamis', 'topee', 'trush', 'shaft', 'trica', 'bogan', 'sexto', 'bronk', 'rivet', 'dixie', 'foism', 'gumly', 'aotus', 'smack', 'withe', 'tabog', 'growl', 'lural', 'comic', 'warty', 'stopa', 'baume', 'broma', 'xylic', 'corke', 'donna', 'grain', 'prion', 'skere', 'payed', 'hawok', 'alite', 'irish', 'luigi', 'stich', 'bahut', 'dicky', 'affix', 'paren', 'chena', 'uplay', 'zapus', 'pooly', 'pride', 'oriya', 'hawky', 'stell', 'sulky', 'logia', 'pubis', 'comid', 'decke', 'rajiv', 'keawe', 'letch', 'annex', 'ascii', 'badge', 'cakey', 'daraf', 'crony', 'potch', 'marco', 'ankle', 

In [9]:
def best_play(good_words, verbose=False):
    """ returns the best play for wordle based on character frequency 
    Note: this is not necessarily optimal since word difficulty needs to be taken into account
     (i.e. 'later' is better then 'artie' because nobody gona put artie in wordle XD)"""
    freq = {}  # get frequency map of all characters
    freq2 = [{} for i in range(wl)]   # get frequency map of character frequency in each position
    for word in good_words:  # TODO: weight freq by how common the word is
        for c in set(word): # save raw freq
            freq[c] = freq.get(c, 0) + 1
        for i, c in enumerate(word):  # save freq for specific places
            freq2[i][c] = freq2[i].get(c, 0) + 1
    if verbose:
        print("*Frequency Map")  # show in sorted order
        print({k: v for k, v in sorted(freq.items(), key=lambda item: item[1], reverse=True)})
        for i in range(len(freq2)):
            print("Frequency2 Map", i)  # show in sorted order
            print({k: v for k, v in sorted(freq2[i].items(), key=lambda item: item[1], reverse=True)})
    
    score = {}  # get score of each word
    for word in good_words:
        sc = 0
        for c in set(word):  # add freq of letter occurring in any word
            sc += freq[c]  # aka chance of -
        for i, c in enumerate(word):  # add freq of letter occurring in that spot in any word (half weighted)
            sc += freq2[i].get(c, 0)/1.5 # aka chance of * 
        score[word] = sc  # TODO: account for previously guessed words
        
    # tunable modifiers to words
    for word in score.keys():
        if word[-1] == "s":  # words ending in s are less frequently used in wordle
            score[word] = score[word] * 0.6
        elif word[-1] == "y":  # words ending in y are slightly less frequently used in wordle
            score[word] = score[word] * 0.8
            
        unq = {}  # words with < 3 unique characters are difficult and are less frequently used in wordle
        for c in word:
            unq[c] = 1
        if len(unq) < 3:
            score[word] = score[word] * 0.7
    # TODO: word freq - penalize words that are super frequent or super esoteric

    # sort possible words based on score
    res = sorted(good_words, key=lambda word: score[word], reverse=True)
    if verbose:
        print("\nWords sorted by best option:")
        print(res)
    return res

ordered_words = best_play(potential_words, True)  # words.split(" ")
play = ordered_words[0]

*Frequency Map
{'a': 967, 'e': 805, 'i': 640, 'r': 622, 'o': 585, 'l': 509, 'n': 508, 's': 504, 't': 497, 'u': 456, 'y': 353, 'c': 340, 'd': 328, 'm': 282, 'p': 278, 'h': 260, 'g': 251, 'b': 244, 'k': 222, 'w': 147, 'f': 117, 'v': 102, 'z': 74, 'j': 54, 'x': 48, 'q': 14}
Frequency2 Map 0
{'s': 237, 'a': 164, 'b': 148, 't': 130, 'c': 129, 'p': 121, 'd': 104, 'm': 100, 'g': 89, 'r': 83, 'l': 76, 'f': 73, 'k': 70, 'u': 62, 'w': 62, 'h': 51, 'n': 51, 'o': 47, 'j': 42, 'e': 41, 'i': 39, 'v': 30, 'z': 19, 'y': 17, 'q': 8, 'x': 3}
Frequency2 Map 1
{'a': 357, 'o': 274, 'e': 221, 'i': 215, 'u': 186, 'r': 160, 'l': 115, 'h': 84, 'n': 82, 't': 44, 'c': 35, 'p': 33, 'm': 31, 'y': 25, 'd': 24, 'w': 24, 's': 17, 'k': 14, 'b': 12, 'g': 11, 'x': 9, 'z': 7, 'v': 7, 'f': 4, 'q': 3, 'j': 2}
Frequency2 Map 2
{'a': 194, 'r': 188, 'n': 167, 'i': 162, 'o': 135, 'e': 118, 'l': 115, 'u': 107, 't': 104, 's': 79, 'm': 78, 'd': 72, 'g': 72, 'c': 68, 'p': 63, 'b': 58, 'v': 35, 'w': 35, 'k': 34, 'y': 25, 'h': 22, '

In [22]:
# Finding the best starting play
words = [word for word in get_english_words_set(['web2'], lower=True) if len(word) == wl]
all_ordered_words = best_play(words, True)
print('\n')
res = {}
for test_word in ["audio", "raise", "ocean"]:  # seeing the strength of proposed words
    res[test_word] = all_ordered_words.index(test_word)*100/len(words)  # top percentile of word according to strength
print(res)

*Frequency Map
{'a': 4798, 'e': 4123, 'r': 3270, 'i': 3053, 'o': 2877, 's': 2608, 'l': 2514, 'n': 2500, 't': 2451, 'u': 2176, 'y': 1755, 'c': 1689, 'd': 1542, 'h': 1506, 'm': 1498, 'p': 1359, 'b': 1233, 'g': 1192, 'k': 1017, 'w': 735, 'f': 647, 'v': 523, 'z': 281, 'x': 232, 'j': 231, 'q': 98}
Frequency2 Map 0
{'s': 1267, 'a': 788, 'c': 732, 'b': 713, 't': 666, 'p': 579, 'm': 518, 'd': 451, 'g': 451, 'r': 436, 'f': 397, 'l': 396, 'h': 307, 'w': 294, 'k': 272, 'u': 270, 'n': 254, 'e': 245, 'o': 210, 'i': 178, 'v': 164, 'j': 156, 'y': 87, 'z': 66, 'q': 60, 'x': 22}
Frequency2 Map 1
{'a': 1752, 'o': 1314, 'e': 1199, 'i': 959, 'u': 867, 'r': 785, 'l': 585, 'h': 534, 'n': 389, 't': 234, 'p': 197, 'c': 172, 'y': 153, 'm': 151, 'w': 134, 's': 110, 'd': 91, 'b': 65, 'k': 65, 'g': 62, 'v': 46, 'x': 39, 'z': 27, 'f': 23, 'q': 14, 'j': 12}
Frequency2 Map 2
{'a': 982, 'r': 957, 'i': 825, 'o': 766, 'n': 752, 'e': 636, 'l': 609, 'u': 538, 't': 502, 's': 410, 'm': 394, 'd': 350, 'b': 303, 'g': 296, 'c

In [18]:
def play_wordle(target, guess):
    """ executes 1 round of wordle and returns feedback """
    assert len(target) == wl and len(guess) == wl
    res = ""
    for i in range(len(target)):
        if target[i] == guess[i]:
            res += '*'
        elif guess[i] in target:
            res += '-'
        else:
            res += 'x'
    return res

def wordle(target, max_guess=6):
    """ executes max_guess rounds of wordle and tells you if you won :) 
    (returns all your guesses & feedback) """
    i = 0
    results = []
    while i <= max_guess:
        inword = str(input("Your Guess: ")).lower().strip()
        feedback = play_wordle(target, inword)
        results.append((inword, feedback))
        print("Feedback: ", feedback)
        print()
        if feedback == '*****':
            print("Congrats! You win :)\n")
            return results
        i += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return results
test = wordle("gummy")

Your Guess: raise
Feedback:  xxxxx

Your Guess: pouty
Feedback:  xx-x*

Your Guess: gummy
Feedback:  *****

Congrats! You win :)



In [23]:
from numpy.random import choice

pool_size = 20  # max deviation from best choice
rand_weight = 1.5  # int [1,inf] exponentially scaling how likely sorted choices are to each other (aka how random)
def bot_wordle(target, num_guesses=6, verbose=False):
    """ shows wordle guesses & results for a freq-optimal bot
    Note: its not very good for reasons discussed earlier """
    counter = 1
    potential_words = words.split(' ')
    guesses = []
    # memory variables between words (can be sets but im lazy)
    good_c = ""
    bad_c = ""
    while counter <= num_guesses:
        assert len(potential_words) > 0, f"No choices found in {potential_words=}"
        
        # random choice of good words is more interesting than always taking the best word
        # tldr masks some of the pitfalls of the strategy in variability
        best_words = best_play(potential_words)
        if len(best_words) > pool_size:  # constrain random choice
            best_words = best_words[:pool_size]
        inword = choice(best_words, 1, [rand_weight**i for i in reversed(range(len(best_words)))])[0]
        
        guesses.append(inword)
        print("Guessed:", inword)
        
        feedback = play_wordle(target, inword)
        print("Feedback:", feedback)
        
        if feedback == '*****':
            print("Congrats! You win :)")
            return
        else:  # find next best option
            to_find = ""
            for i, c in enumerate(feedback):  # update params based on feedback
                if c == "*":
                    to_find += inword[i]
                else:
                    to_find += '.'
                    if c == "-":
                        good_c += inword[i]
                    else:
                        bad_c += inword[i]
            # update words for next round
            if verbose: print("Update:", ', '.join([to_find, bad_c, good_c]))
            potential_words = match_words(to_find, bad_c, good_c, guesses)
        print()
        counter += 1
    print("Sorry thats not right...\nThe correct word was:", target)
    return

bot_wordle("mulch", 6, True)

# TODO: make it so bot looks at previous guesses and knows - letters can be in that position*

Guessed: raise
Feedback: xxxxx
Update: ....., raise, 

Guessed: boldu
Feedback: xx*x-
Update: ..l.., raisebod, u

Guessed: fully
Feedback: x**-x
Update: .ul.., raisebodfy, ul

Guessed: mulch
Feedback: *****
Congrats! You win :)
